# RoBERTa Machine Generated Text Classifier

Follows:
> Adaku Uchendu, Thai Le, Kai Shu, and Dongwon Lee. 2020. [Authorship Attribution for Neural Text Generation](https://aclanthology.org/2020.emnlp-main.673/). In Proceedings of the 2020 Conference on Empirical Methods in Natural Language Processing (EMNLP), pages 8384–8395, Online. Association for Computational Linguistics.

Download training data from [here](https://raw.githubusercontent.com/AdaUchendu/Authorship-Attribution-for-Neural-Text-Generation/master/data/balanced_p2.csv).

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from transformers import set_seed
from transformers import RobertaForSequenceClassification, RobertaTokenizer, Trainer, TrainingArguments,AutoModelForSequenceClassification, DataCollatorWithPadding
import torch
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
balanced_df = pd.read_csv('balanced_p2.csv', index_col=0).reset_index(drop=True)
balanced_df.rename(columns={'class': 'label'}, inplace=True)
train_df, test_df = train_test_split(balanced_df, test_size=0.2, random_state=13, stratify=balanced_df['label'])

In [ ]:
train_df['label'].value_counts()

In [ ]:
test_df['label'].value_counts()

In [ ]:
# Prepare training and validation data
train_texts = train_df.text.tolist()
train_labels = train_df.label.tolist()
val_texts = test_df.text.tolist()
val_labels = test_df.label.tolist()

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='macro')
    return {
        'accuracy': acc,
        'f1': f1,
        'eval_accuracy': acc
    }

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=512, pad_to_multiple_of=None)

In [ ]:
training_args = TrainingArguments(
    output_dir='./models',
    num_train_epochs=10 ,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="eval_f1",
    load_best_model_at_end=True,
    save_total_limit=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16
)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
        
    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,    
)

trainer.train()

In [ ]:
trainer.state.best_model_checkpoint, trainer.state.best_metric

In [ ]:
# Load the trained model
model = AutoModelForSequenceClassification.from_pretrained(trainer.state.best_model_checkpoint)
# Set the model to evaluation mode
model.eval()


In [ ]:
def predict(input_text):
    # Tokenize the input text
    input_encodings = tokenizer(input_text, truncation=True, padding=True, return_tensors="pt")

    # Generate predictions for the input text
    with torch.no_grad():
        logits = model(**input_encodings).logits

    # Get the predicted label
    pred_label = logits.argmax().item()
    pred_prob = logits.softmax(dim=1).max().item()
    return pred_label, pred_prob


# Define the input text to be classified
input_text = "Text to test"
predict(input_text)


In [ ]:
general_df = pd.read_csv("../priv_data/chatgpt-general.csv", index_col=0)
general_df['source'] = 'chatgpt-general'
topic_df = pd.read_csv("../priv_data/chatgpt-topic.csv", index_col=0)
topic_df['source'] = 'chatgpt-topic'
claimreview_df = pd.read_csv("../priv_data/claimreview.csv", index_col=0)
claimreview_df['source'] = 'claimreview'

all_df = pd.concat([general_df, topic_df, claimreview_df], ignore_index=True)

Labels are human=0, machine=1.
See cell 56 in [this notebook](https://github.com/AdaUchendu/Authorship-Attribution-for-Neural-Text-Generation/blob/dd0ed003dc4db7979f34f9480636d4417d9a827a/Clean_Generated/Single/P2%20Binary%20-%20Human%20vs%20Machine.ipynb)

In [ ]:
all_df['true_label'] = all_df['source'].apply(lambda x: 0 if x == 'claimreview' else 1)

In [ ]:
all_df['true_label'].value_counts()

In [ ]:
predictions=[]
probs = []
for claim in tqdm(all_df['claim'].tolist()):
    label, prob = predict(claim)
    predictions.append(label)
    probs.append(prob)

In [ ]:
all_df['pred'] = predictions
all_df['prob'] = probs

In [ ]:
all_df['pred'].value_counts()

In [ ]:
r_general_df = all_df[all_df['source'] == 'chatgpt-general']
f1_score(r_general_df['true_label'] == 1, r_general_df['pred'] == 1, average='binary')

In [ ]:
r_topic_df = all_df[all_df['source'] == 'chatgpt-topic']
f1_score(r_topic_df['true_label'] == 1, r_topic_df['pred'] == 1, average='binary')

In [ ]:
r_cr_df = all_df[all_df['source'] == 'claimreview']
f1_score(r_cr_df['true_label'] == 0, r_cr_df['pred'] == 0, average='binary')

In [ ]:
f1_score(all_df['true_label'], all_df['pred'], average='macro')

In [ ]:
all_df['prob'].describe()